In [ ]:
import os
import pandas as pd

In [22]:
EVENT_FOLDER = './events_csv'
EVENT_FILES = os.listdir(EVENT_FOLDER)

event_list = []
event_names = ['Dallas', 'New Orleans', 'Atlanta', 'Birmingham', ' Pro League 1', 'Relegation', 'Seattle',
               'Anaheim', 'Pro League 2', 'Champs']
event_dict = {}
ww2_snd = []

for x in EVENT_FILES:
    with open('{}/{}'.format(EVENT_FOLDER, x)) as f:
        temp = pd.read_csv(f)
        event_list.append(temp)

event_dict = dict(zip(event_names, event_list))

meta_df = pd.concat(event_list[:])

ww2_snd = meta_df[meta_df['mode'] == 'Search & Destroy']

In [46]:
w_df = ww2_snd[:]

# stats to be used in model
target_stats = ['match id','player', 'fave weapon', 'kills', 'kills (stayed alive)', 'shots', 
                'avg time per life (s)', 'snd firstbloods', 'scorestreaks earned', 'snd rounds']

# objects within 'fave weapon' that influence model results
n2low = ['M1 Garand', 'M1941']
distort = ['Kar98k', 'Springfield'] + n2low


w_df2 = w_df[target_stats]

w_df3 = w_df2[~w_df2['fave weapon'].isin(distort)].copy()
w_df3['avg time per life (s)'] = pd.to_numeric(w_df3['avg time per life (s)'], errors="coerce")

stat_loop = ['fave weapon', 'kills', 'kills (stayed alive)', 'shots', 'avg time per life (s)', 
             'snd firstbloods', 'scorestreaks earned']
per_rnd_avgs = ['akpr', 'antk', 'avs', 'atpl', 'afbr', 'ser']

# need to make into a loop but was getting the following error:
#  unsupported operand type(s) for /: 'str' and 'int'
w_df3['akpr'] = w_df3['kills']/w_df3['snd rounds']
w_df3['antk'] = w_df3['kills (stayed alive)']/w_df3['snd rounds']
w_df3['avs'] = w_df3['shots']/w_df3['snd rounds']
w_df3['atpl'] = w_df3['avg time per life (s)']/w_df3['snd rounds']
w_df3['afbr'] = w_df3['snd firstbloods']/w_df3['snd rounds']
w_df3['ser'] = w_df3['scorestreaks earned']/w_df3['snd rounds']


In [66]:
by_weapon_per_stat_means = w_df3.groupby('fave weapon')['akpr', 'antk', 'avs', 'atpl', 'afbr', 'ser'].mean()

print(by_weapon_per_stat_means)

stat_mean = by_weapon_per_stat_means.mean()

by_weapon_per_stat_means['akpr'] = (by_weapon_per_stat_means['akpr']-stat_mean[0])/stat_mean[0]
by_weapon_per_stat_means['antk'] = (by_weapon_per_stat_means['antk']-stat_mean[1])/stat_mean[1]
by_weapon_per_stat_means['avs'] = (by_weapon_per_stat_means['avs']-stat_mean[2])/stat_mean[2]
by_weapon_per_stat_means['atpl'] = (by_weapon_per_stat_means['atpl']-stat_mean[3])/stat_mean[3]
by_weapon_per_stat_means['afbr'] = (by_weapon_per_stat_means['afbr']-stat_mean[4])/stat_mean[4]
by_weapon_per_stat_means['ser'] = (by_weapon_per_stat_means['ser']-stat_mean[5])/stat_mean[5]

pct_adj_final = by_weapon_per_stat_means[:]
print(pct_adj_final)


                 akpr      antk        avs      atpl      afbr       ser
fave weapon                                                             
BAR          0.707278  0.581772  14.393547  4.976426  0.109711  0.022617
FG 42        0.718197  0.605839  11.807250  5.294200  0.118996  0.041071
PPSh-41      0.678473  0.550985  14.490083  4.680116  0.130141  0.025398
STG-44       0.691446  0.577466  17.317993  4.956471  0.115907  0.027732
                 akpr      antk       avs      atpl      afbr       ser
fave weapon                                                            
BAR          0.012062  0.004761 -0.007493 -0.000076 -0.075640 -0.225562
FG 42        0.027687  0.046326 -0.185831  0.063775  0.002587  0.406311
PPSh-41     -0.029156 -0.048411 -0.000837 -0.059614  0.096491 -0.130327
STG-44      -0.010593 -0.002676  0.194162 -0.004085 -0.023439 -0.050421
